In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error as mse
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential,Model
from keras.layers import Input,LSTM, Dense, Flatten, Conv1D, Lambda, Reshape
from keras.layers.merge import concatenate, multiply,add
import tensorflow as tf
from keras import regularizers
from keras.initializers import glorot_uniform
from tqdm import tqdm
from keras import regularizers
from keras.models import load_model
from statsmodels.tsa.seasonal import seasonal_decompose
import datetime
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler


import rpy2
import rpy2.robjects.numpy2ri
from stldecompose import decompose
rpy2.robjects.numpy2ri.activate()
from rpy2.robjects.packages import importr
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()
stats = importr('stats')
stl=stats.stl
ts =stats.ts

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
data= pd.read_csv("Data/FromMohammed/cif_dataset_complete.csv",header=None)
predictions = pd.read_csv("Data/FromMohammed/theta_25_horg.csv",index_col=0,skiprows = [1])
print(data.shape)
print(predictions.shape)

(72, 123)
(72, 90)


In [3]:
def make_input(data,window_size,horizon=1):
    length=data.shape[0]
#     depth=data.shape[2]
    y = np.zeros([length-window_size+1-horizon,horizon])
    output=np.zeros([length-window_size+1-horizon,window_size])
    for i in range(length-window_size+1-horizon):
        output[i:i+1,:]=data[i:i+window_size]
        y[i,:]= data[i+window_size:i+window_size+horizon]
    return output.reshape(output.shape[0],window_size,1), y
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def make_k_input(data,window_size,horizon):
    length = data.shape[0]
    output= np.zeros([length-window_size+1-horizon,horizon])
    for i in range(length-window_size-horizon+1):
        output[i:i+1,:]=data[i+window_size:i+window_size+horizon]
    return output.reshape(output.shape[0],horizon,1)
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def nonov_make_input(data,window_size,horizon=1):
    length=data.shape[0]-window_size
    loop=length//horizon
    extra = length%horizon
#     print(str(extra))
    data = np.append(data,np.zeros([horizon-extra]))
#     print(data)
    if extra ==0:
        i_val = loop
    else:
        i_val=loop+1
        
    output=np.zeros([i_val,window_size])
    y=np.zeros([i_val,horizon])
    for i in range(i_val):
        output[i:i+1,:]=data[i*horizon:(i*horizon)+window_size]
        y[i,:]= data[(i*horizon)+window_size:(i*horizon)+window_size+horizon]
        
    return output.reshape(output.shape[0],window_size,1), y
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def nonov_make_k_input(data,window_size,horizon):
    length = data.shape[0]-window_size
    loop=length//horizon
    extra = length%horizon
    data_app = np.repeat(data[-1],extra)
    data = np.append(data,data_app)    
#     data = np.append(data,np.zeros([horizon-extra]))
    if extra ==0:
        i_val = loop
    else:
        i_val=loop+1
    output=np.zeros([i_val,horizon])
    for i in range(i_val):
        output[i:i+1,:]=data[(i*horizon)+window_size:(i*horizon)+window_size+horizon]
    return output.reshape(output.shape[0],horizon,1)    

In [4]:
def smape(val,preds,horizon):
    temp = np.abs(val-preds)
    temp1=np.abs(val)+np.abs(preds)
    
    smape = 200/horizon*np.sum(temp/temp1)

    return smape




def smape_nzr(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred))
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return 200.0 * np.mean(diff)

# Case 1 : Only Data
### Horizon=6   ///   num_test=6,12

In [96]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,12])
    final_test = np.zeros([data_length,12])
    Final_smape1=np.zeros(data_length)
    for y in range(data_length):
        
        final_predictions = np.zeros([data_length,12])
        final_test = np.zeros([data_length,12])
        num_test = data.iloc[y].values[1]
        horizon=6
        window_size=6
        current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-num_test]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
        
        if n_val < horizon: #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            n_val = horizon#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            
            
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################
        LL=series_data.shape[0]-n_val
        HH=LL-window_size+1-horizon
        if (HH<0):
            train = series_data[:-n_val+1-HH]
        else:
            train = series_data[:-n_val]                        
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################

        #train = series_data[:-n_val+3]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        test = series_d[-(num_test+window_size):]
        val = series_data[-(n_val+window_size):]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=series_d[-num_test:]


        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]
        
        #x_test = test_sequence_norm[0].reshape(1,test_sequence_norm[0].shape[0]*test_sequence_norm[0].shape[1],1)
        #y_test = test_sequence_norm[1].reshape(1,test_sequence_norm[1].shape[0]*test_sequence_norm[1].shape[1])
        
        
        
############################################################################################################
############################################################################################################
############################################################################################################
        train_input = x_train
        val_input = x_val
        test_input = x_test

####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        tf.reset_default_graph()
        K.clear_session()
        
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')

        

        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_1 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        branch_2 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)
        

        model=Model(inputs=[input_data],outputs=net)
        if y==71:
            model.summary()
        callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF01.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.01))


        model.fit({'input_data':train_input},y_train,validation_data=[[val_input],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)

        model.load_weights('MyModelCheckpoint/CIF01.h5')
        pred=model.predict({'input_data':test_input})
        pred=scaler3.inverse_transform(pred)#######################################################
        #pred.reshape(1,pred.shape[0]*pred.shape[1])

        final_predictions[y,:num_test] = pred.reshape(num_test)
        final_test[y,:num_test]=test[-num_test:]

        AAA=smape(pred.reshape(num_test),current_test,num_test)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(num_test),current_test)
        Final_smape1[y]=AAA
        mse1+=mse(pred.reshape(num_test),current_test)
        pbar.update(1)

np.savetxt('MyModelCheckpoint/prediction01.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/72)
print("SMAPE2 (NZR approach)")
print(smape2/72)
print("MSE1:")
print(mse1/72)
print (sum(Final_smape1)/72)    
#print(smape1)
#print(smape2)

2019-08-22 01:07:30.008657


 99%|████████████████████████████████████████████████████████████████████████████████▊ | 71/72 [01:58<00:01,  1.33s/it]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_data (InputLayer)      (None, 6, 1)              0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 6, 32)             128       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 6, 32)             3104      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 6, 32)             3104      
_________________________________________________________________
flatten_1 (Flatten)          (None, 192)               0         
_________________________________________________________________
dense_final (Dense)          (None, 6)                 1158      
Total params: 7,494
Trainable params: 7,494
Non-trainable params: 0
_________________________________________________________________


100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [02:00<00:00,  1.35s/it]


2019-08-22 01:09:30.172727
-----------------
SMAPE1: (DeepEX approach)
13.179894760175314
SMAPE2 (NZR approach)
13.179894760175312
MSE1:
5043259741287.838
13.179894760175314


# Case 2: Adding Expert Knowledge to layer 1
### Horizon=6   ///   num_test=6,12
### Using Concatenation

In [99]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,12])
    final_test = np.zeros([data_length,12])
    Final_smape2=np.zeros(data_length)
    for y in range(data_length):
        
        final_predictions = np.zeros([data_length,12])
        final_test = np.zeros([data_length,12])
        num_test = data.iloc[y].values[1]
        horizon=6
        window_size=6
        current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-num_test]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
        
        if n_val < horizon: #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            n_val = horizon#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            
            
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################
        LL=series_data.shape[0]-n_val
        HH=LL-window_size+1-horizon
        if (HH<0):
            train = series_data[:-n_val+1-HH]
        else:
            train = series_data[:-n_val]                        
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################

        #train = series_data[:-n_val+3]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        test = series_d[-(num_test+window_size):]
        val = series_data[-(n_val+window_size):]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=series_d[-num_test:]


        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]
        
############################################################################################################
############################################################################################################
############################################################################################################
        train_input = x_train
        val_input = x_val
        test_input = x_test

####################################################################################
#######creating train, validation and test sets from Predictions####################
####################################################################################
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
#####################################################################
        series_p=current_pred
        series_pred=series_p[:-num_test]
        
        if (HH<0):
            train_p = series_pred[:-n_val+1-HH]
        else:
            train_p = series_pred[:-n_val]                        
        
        
        #train_p = series_pred[:-n_val+3]
        val_p = series_pred[-(n_val+window_size):]
        test_p = series_p[-(num_test+window_size):]

        train_pred = make_k_input(train_p,window_size,horizon)
        val_pred = make_k_input(val_p,window_size,horizon)
        test_pred = nonov_make_k_input(test_p,window_size,horizon)        
        
        
        train_pred_norm=np.zeros(train_pred.shape)
        val_pred_norm=np.zeros(val_pred.shape)
        test_pred_norm=np.zeros(test_pred.shape)
    
        for j in range(train_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(train_pred[j])
            train_pred_norm[j]= scaler.transform(train_pred[j])
        for j in range(val_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(val_pred[j])
            val_pred_norm[j]= scaler.transform(val_pred[j])
        for j in range(test_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(test_pred[j])
            test_pred_norm[j]= scaler.transform(test_pred[j])
        
        train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],train_pred_norm.shape[1])
        val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],val_pred_norm.shape[1])
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],test_pred_norm.shape[1])

##########################################################################################################################
        ncol = 6
        encoding_dim = 4
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        #encoder = Model(inputs = input_dim, outputs = encoded2)
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('MyModelCheckpoint/encoder_model.h5')
        train_pred_norm_auto=np.array(encoder.predict(train_pred_norm))
        val_pred_norm_auto=np.array(encoder.predict(val_pred_norm))
        test_pred_auto=np.array(encoder.predict(test_pred_norm))

        train_pred_norm_auto=pd.DataFrame(train_pred_norm_auto)
        val_pred_norm_auto=pd.DataFrame(val_pred_norm_auto)
        test_pred_auto=pd.DataFrame(test_pred_auto)                    
        
        

####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        tf.reset_default_graph()
        K.clear_session()
        
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        input_pred=Input(batch_shape=(None,4),name='input_pred')
        
        encoded12 = Dense(8, activation = 'relu')(input_pred)
        encoded22 = Dense(16, activation = 'relu')(encoded12)
        encoded32 = Dense(32, activation = 'relu')(encoded22)
        encoded32=Reshape((1,32))(encoded32)
        

        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_0=concatenate([branch_0,encoded32],axis=1)
        #branch_0=add([branch_0,encoded32])
        branch_1 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        #branch_1=add([branch_1,encoded32])
        branch_2 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        #branch_2=add([branch_2,encoded32])
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)
        

        model=Model(inputs=[input_data,input_pred],outputs=net)
        if y==71:
            model.summary()
        callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF02.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.01))


        model.fit({'input_data':train_input,'input_pred':train_pred_norm_auto},y_train,validation_data=[[val_input,val_pred_norm_auto],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)

        model.load_weights('MyModelCheckpoint/CIF02.h5')
        
        pred=model.predict({'input_data':test_input, 'input_pred':test_pred_auto})
        pred=scaler3.inverse_transform(pred)#######################################################

        
        final_predictions[y,:num_test] = pred.reshape(num_test)
        final_test[y,:num_test]=test[-num_test:]

        AAA=smape(pred.reshape(num_test),current_test,num_test)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(num_test),current_test)
        Final_smape2[y]=AAA
        mse1+=mse(pred.reshape(num_test),current_test)
        pbar.update(1)

np.savetxt('MyModelCheckpoint/prediction02.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/72)
print("SMAPE2 (NZR approach)")
print(smape2/72)
print("MSE1:")
print(mse1/72)
print (sum(Final_smape2)/72)    
#print(smape1)
#print(smape2)

2019-08-22 01:29:13.740692


  0%|                                                                                           | 0/72 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  1%|█▏                                                                                 | 1/72 [00:02<02:57,  2.50s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  3%|██▎                                                                                | 2/72 [00:05<02:58,  2.55s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 32%|██████████████████████████▏                                                       | 23/72 [01:00<02:12,  2.71s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 33%|███████████████████████████▎                                                      | 24/72 [01:03<02:09,  2.70s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 35%|████████████████████████████▍                                                     | 25/72 [01:06<02:06,  2.70s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 64%|████████████████████████████████████████████████████▍                             | 46/72 [02:01<01:06,  2.57s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 65%|█████████████████████████████████████████████████████▌                            | 47/72 [02:03<01:04,  2.58s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 67%|██████████████████████████████████████████████████████▋                           | 48/72 [02:06<01:02,  2.60s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 96%|██████████████████████████████████████████████████████████████████████████████▌   | 69/72 [02:48<00:06,  2.01s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 97%|███████████████████████████████████████████████████████████████████████████████▋  | 70/72 [02:50<00:04,  2.00s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 99%|████████████████████████████████████████████████████████████████████████████████▊ | 71/72 [02:52<00:01,  1.99s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_pred (InputLayer)         (None, 4)            0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 8)            40          input_pred[0][0]                 
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 16)           144         dense_1[0][0]                    
__________________________________________________________________________________________________
input_data (InputLayer)         (None, 6, 1)         0                                            
__________________________________________________________________________________________________
dense_3 (D

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [02:54<00:00,  1.99s/it]


2019-08-22 01:32:08.204856
-----------------
SMAPE1: (DeepEX approach)
12.488783797089791
SMAPE2 (NZR approach)
12.488783797089788
MSE1:
5520113106953.034
12.488783797089791


# Case 3:
### Data + Expert Knowledge (Size 32)
### Adding Expert Knowledge to Layer 1
#### Conv1D(32)
#### Conv1D(64)
#### Conv1D(64)

In [37]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,12])
    final_test = np.zeros([data_length,12])
    Final_smape2=np.zeros(data_length)
    for y in range(data_length):
        
        final_predictions = np.zeros([data_length,12])
        final_test = np.zeros([data_length,12])
        num_test = data.iloc[y].values[1]
        horizon=6
        window_size=6
        current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-num_test]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
        
        if n_val < horizon: #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            n_val = horizon#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            
            
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################
        LL=series_data.shape[0]-n_val
        HH=LL-window_size+1-horizon
        if (HH<0):
            train = series_data[:-n_val+1-HH]
        else:
            train = series_data[:-n_val]                        
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################

        #train = series_data[:-n_val+3]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        test = series_d[-(num_test+window_size):]
        val = series_data[-(n_val+window_size):]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=series_d[-num_test:]


        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]
        
############################################################################################################
############################################################################################################
############################################################################################################
        train_input = x_train
        val_input = x_val
        test_input = x_test

####################################################################################
#######creating train, validation and test sets from Predictions####################
####################################################################################
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
#####################################################################
        series_p=current_pred
        series_pred=series_p[:-num_test]
        
        if (HH<0):
            train_p = series_pred[:-n_val+1-HH]
        else:
            train_p = series_pred[:-n_val]                        
        
        
        #train_p = series_pred[:-n_val+3]
        val_p = series_pred[-(n_val+window_size):]
        test_p = series_p[-(num_test+window_size):]

        train_pred = make_k_input(train_p,window_size,horizon)
        val_pred = make_k_input(val_p,window_size,horizon)
        test_pred = nonov_make_k_input(test_p,window_size,horizon)        
        
        
        train_pred_norm=np.zeros(train_pred.shape)
        val_pred_norm=np.zeros(val_pred.shape)
        test_pred_norm=np.zeros(test_pred.shape)
    
        for j in range(train_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(train_pred[j])
            train_pred_norm[j]= scaler.transform(train_pred[j])
        for j in range(val_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(val_pred[j])
            val_pred_norm[j]= scaler.transform(val_pred[j])
        for j in range(test_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(test_pred[j])
            test_pred_norm[j]= scaler.transform(test_pred[j])
        
        train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],train_pred_norm.shape[1])
        val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],val_pred_norm.shape[1])
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],test_pred_norm.shape[1])

##########################################################################################################################
        ncol = 6
        encoding_dim = 4
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        #encoder = Model(inputs = input_dim, outputs = encoded2)
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('MyModelCheckpoint/encoder_model.h5')
        train_pred_norm_auto=np.array(encoder.predict(train_pred_norm))
        val_pred_norm_auto=np.array(encoder.predict(val_pred_norm))
        test_pred_auto=np.array(encoder.predict(test_pred_norm))

        train_pred_norm_auto=pd.DataFrame(train_pred_norm_auto)
        val_pred_norm_auto=pd.DataFrame(val_pred_norm_auto)
        test_pred_auto=pd.DataFrame(test_pred_auto)                    
        
        

####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        tf.reset_default_graph()
        K.clear_session()
        
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        input_pred=Input(batch_shape=(None,4),name='input_pred')
        
        encoded12 = Dense(8, activation = 'relu')(input_pred)
        encoded22 = Dense(16, activation = 'relu')(encoded12)
        encoded32 = Dense(32, activation = 'relu')(encoded22)
        encoded32=Reshape((1,32))(encoded32)
        

        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_0=concatenate([branch_0,encoded32],axis=1)
        #branch_0=add([branch_0,encoded32])
        branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        #branch_1=add([branch_1,encoded32])
        branch_2 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        #branch_2=add([branch_2,encoded32])
        
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.01))(branch_3)
        

        model=Model(inputs=[input_data,input_pred],outputs=net)
        if y==71:
            model.summary()
        callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF02.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.01))


        model.fit({'input_data':train_input,'input_pred':train_pred_norm_auto},y_train,validation_data=[[val_input,val_pred_norm_auto],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)

        model.load_weights('MyModelCheckpoint/CIF02.h5')
        
        pred=model.predict({'input_data':test_input, 'input_pred':test_pred_auto})
        pred=scaler3.inverse_transform(pred)#######################################################

        
        final_predictions[y,:num_test] = pred.reshape(num_test)
        final_test[y,:num_test]=test[-num_test:]

        AAA=smape(pred.reshape(num_test),current_test,num_test)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(num_test),current_test)
        Final_smape2[y]=AAA
        mse1+=mse(pred.reshape(num_test),current_test)
        pbar.update(1)

np.savetxt('MyModelCheckpoint/prediction02.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/72)
print("SMAPE2 (NZR approach)")
print(smape2/72)
print("MSE1:")
print(mse1/72)
print (sum(Final_smape2)/72)    
#print(smape1)
#print(smape2)

2019-08-31 19:44:37.865438


  0%|                                                                                           | 0/72 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  1%|█▏                                                                                 | 1/72 [00:02<03:10,  2.69s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  3%|██▎                                                                                | 2/72 [00:05<03:05,  2.65s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 32%|██████████████████████████▏                                                       | 23/72 [01:01<02:13,  2.72s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 33%|███████████████████████████▎                                                      | 24/72 [01:03<02:10,  2.73s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 35%|████████████████████████████▍                                                     | 25/72 [01:06<02:07,  2.72s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 64%|████████████████████████████████████████████████████▍                             | 46/72 [02:02<01:09,  2.66s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 65%|█████████████████████████████████████████████████████▌                            | 47/72 [02:05<01:07,  2.69s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 67%|██████████████████████████████████████████████████████▋                           | 48/72 [02:07<01:04,  2.68s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 96%|██████████████████████████████████████████████████████████████████████████████▌   | 69/72 [02:50<00:06,  2.03s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 97%|███████████████████████████████████████████████████████████████████████████████▋  | 70/72 [02:52<00:04,  2.03s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 99%|████████████████████████████████████████████████████████████████████████████████▊ | 71/72 [02:54<00:02,  2.01s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_pred (InputLayer)         (None, 4)            0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 8)            40          input_pred[0][0]                 
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 16)           144         dense_1[0][0]                    
__________________________________________________________________________________________________
input_data (InputLayer)         (None, 6, 1)         0                                            
__________________________________________________________________________________________________
dense_3 (D

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [02:56<00:00,  2.04s/it]


2019-08-31 19:47:34.608848
-----------------
SMAPE1: (DeepEX approach)
11.40457357906134
SMAPE2 (NZR approach)
11.404573579061337
MSE1:
5425670020031.992
11.40457357906134


# Case 4:
### Data + Expert Knowledge (Size 64)
### Adding Expert Knowledge to Layer 1
#### Conv1D(64)
#### Conv1D(64)
#### Conv1D(64)

In [13]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,12])
    final_test = np.zeros([data_length,12])
    Final_smape2=np.zeros(data_length)
    for y in range(data_length):
        
        final_predictions = np.zeros([data_length,12])
        final_test = np.zeros([data_length,12])
        num_test = data.iloc[y].values[1]
        horizon=6
        window_size=6
        current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-num_test]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
        
        if n_val < horizon: #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            n_val = horizon#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            
            
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################
        LL=series_data.shape[0]-n_val
        HH=LL-window_size+1-horizon
        if (HH<0):
            train = series_data[:-n_val+1-HH]
        else:
            train = series_data[:-n_val]                        
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################

        #train = series_data[:-n_val+3]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        test = series_d[-(num_test+window_size):]
        val = series_data[-(n_val+window_size):]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=series_d[-num_test:]


        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]
        
############################################################################################################
############################################################################################################
############################################################################################################
        train_input = x_train
        val_input = x_val
        test_input = x_test

####################################################################################
#######creating train, validation and test sets from Predictions####################
####################################################################################
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
#####################################################################
        series_p=current_pred
        series_pred=series_p[:-num_test]
        
        if (HH<0):
            train_p = series_pred[:-n_val+1-HH]
        else:
            train_p = series_pred[:-n_val]                        
        
        
        #train_p = series_pred[:-n_val+3]
        val_p = series_pred[-(n_val+window_size):]
        test_p = series_p[-(num_test+window_size):]

        train_pred = make_k_input(train_p,window_size,horizon)
        val_pred = make_k_input(val_p,window_size,horizon)
        test_pred = nonov_make_k_input(test_p,window_size,horizon)        
        
        
        train_pred_norm=np.zeros(train_pred.shape)
        val_pred_norm=np.zeros(val_pred.shape)
        test_pred_norm=np.zeros(test_pred.shape)
    
        for j in range(train_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(train_pred[j])
            train_pred_norm[j]= scaler.transform(train_pred[j])
        for j in range(val_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(val_pred[j])
            val_pred_norm[j]= scaler.transform(val_pred[j])
        for j in range(test_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(test_pred[j])
            test_pred_norm[j]= scaler.transform(test_pred[j])
        
        train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],train_pred_norm.shape[1])
        val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],val_pred_norm.shape[1])
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],test_pred_norm.shape[1])

##########################################################################################################################
        ncol = 6
        encoding_dim = 4
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        #encoder = Model(inputs = input_dim, outputs = encoded2)
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('MyModelCheckpoint/encoder_model.h5')
        train_pred_norm_auto=np.array(encoder.predict(train_pred_norm))
        val_pred_norm_auto=np.array(encoder.predict(val_pred_norm))
        test_pred_auto=np.array(encoder.predict(test_pred_norm))

        train_pred_norm_auto=pd.DataFrame(train_pred_norm_auto)
        val_pred_norm_auto=pd.DataFrame(val_pred_norm_auto)
        test_pred_auto=pd.DataFrame(test_pred_auto)                    
        
        

####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        tf.reset_default_graph()
        K.clear_session()
        
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        input_pred=Input(batch_shape=(None,4),name='input_pred')
        
        encoded12 = Dense(8, activation = 'relu')(input_pred)
        encoded22 = Dense(16, activation = 'relu')(encoded12)
        encoded32 = Dense(32, activation = 'relu')(encoded22)
        encoded42 = Dense(64, activation = 'relu')(encoded32)
        encoded42=Reshape((1,64))(encoded42)
        

        branch_0 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_0=concatenate([branch_0,encoded42],axis=1)
        #branch_0=add([branch_0,encoded32])
        branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        #branch_1=add([branch_1,encoded32])
        branch_2 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        #branch_2=add([branch_2,encoded32])
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.01))(branch_3)
        

        model=Model(inputs=[input_data,input_pred],outputs=net)
        if y==71:
            model.summary()
        callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF028.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.01))


        model.fit({'input_data':train_input,'input_pred':train_pred_norm_auto},y_train,validation_data=[[val_input,val_pred_norm_auto],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)

        model.load_weights('MyModelCheckpoint/CIF028.h5')
        
        pred=model.predict({'input_data':test_input, 'input_pred':test_pred_auto})
        pred=scaler3.inverse_transform(pred)#######################################################

        
        final_predictions[y,:num_test] = pred.reshape(num_test)
        final_test[y,:num_test]=test[-num_test:]

        AAA=smape(pred.reshape(num_test),current_test,num_test)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(num_test),current_test)
        Final_smape2[y]=AAA
        mse1+=mse(pred.reshape(num_test),current_test)
        pbar.update(1)

np.savetxt('MyModelCheckpoint/prediction028.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/72)
print("SMAPE2 (NZR approach)")
print(smape2/72)
print("MSE1:")
print(mse1/72)
print (sum(Final_smape2)/72)    
#print(smape1)
#print(smape2)

2019-08-31 22:20:47.518504


  0%|                                                                                           | 0/72 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  1%|█▏                                                                                 | 1/72 [00:02<03:12,  2.72s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  3%|██▎                                                                                | 2/72 [00:05<03:15,  2.79s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 32%|██████████████████████████▏                                                       | 23/72 [01:07<02:27,  3.02s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 33%|███████████████████████████▎                                                      | 24/72 [01:10<02:24,  3.02s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 35%|████████████████████████████▍                                                     | 25/72 [01:13<02:21,  3.00s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 64%|████████████████████████████████████████████████████▍                             | 46/72 [02:14<01:15,  2.92s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 65%|█████████████████████████████████████████████████████▌                            | 47/72 [02:17<01:13,  2.92s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 67%|██████████████████████████████████████████████████████▋                           | 48/72 [02:20<01:09,  2.91s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 96%|██████████████████████████████████████████████████████████████████████████████▌   | 69/72 [03:07<00:06,  2.25s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 97%|███████████████████████████████████████████████████████████████████████████████▋  | 70/72 [03:09<00:04,  2.21s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 99%|████████████████████████████████████████████████████████████████████████████████▊ | 71/72 [03:11<00:02,  2.16s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_pred (InputLayer)         (None, 4)            0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 8)            40          input_pred[0][0]                 
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 16)           144         dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 32)           544         dense_2[0][0]                    
__________________________________________________________________________________________________
input_data

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [03:13<00:00,  2.20s/it]


2019-08-31 22:24:01.128663
-----------------
SMAPE1: (DeepEX approach)
11.723996332635313
SMAPE2 (NZR approach)
11.723996332635311
MSE1:
3603258568866.917
11.723996332635313


# Case 5:
### Data + Expert Knowledge (Size 32)
### Adding Expert Knowledge to Layer 1
#### Conv1D(32)
#### Conv1D(64)
#### Conv1D(128)

In [20]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,12])
    final_test = np.zeros([data_length,12])
    Final_smape2=np.zeros(data_length)
    for y in range(data_length):
        
        final_predictions = np.zeros([data_length,12])
        final_test = np.zeros([data_length,12])
        num_test = data.iloc[y].values[1]
        horizon=6
        window_size=6
        current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-num_test]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
        
        if n_val < horizon: #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            n_val = horizon#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            
            
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################
        LL=series_data.shape[0]-n_val
        HH=LL-window_size+1-horizon
        if (HH<0):
            train = series_data[:-n_val+1-HH]
        else:
            train = series_data[:-n_val]                        
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################

        #train = series_data[:-n_val+3]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        test = series_d[-(num_test+window_size):]
        val = series_data[-(n_val+window_size):]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=series_d[-num_test:]


        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]
        
############################################################################################################
############################################################################################################
############################################################################################################
        train_input = x_train
        val_input = x_val
        test_input = x_test

####################################################################################
#######creating train, validation and test sets from Predictions####################
####################################################################################
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
#####################################################################
        series_p=current_pred
        series_pred=series_p[:-num_test]
        
        if (HH<0):
            train_p = series_pred[:-n_val+1-HH]
        else:
            train_p = series_pred[:-n_val]                        
        
        
        #train_p = series_pred[:-n_val+3]
        val_p = series_pred[-(n_val+window_size):]
        test_p = series_p[-(num_test+window_size):]

        train_pred = make_k_input(train_p,window_size,horizon)
        val_pred = make_k_input(val_p,window_size,horizon)
        test_pred = nonov_make_k_input(test_p,window_size,horizon)        
        
        
        train_pred_norm=np.zeros(train_pred.shape)
        val_pred_norm=np.zeros(val_pred.shape)
        test_pred_norm=np.zeros(test_pred.shape)
    
        for j in range(train_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(train_pred[j])
            train_pred_norm[j]= scaler.transform(train_pred[j])
        for j in range(val_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(val_pred[j])
            val_pred_norm[j]= scaler.transform(val_pred[j])
        for j in range(test_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(test_pred[j])
            test_pred_norm[j]= scaler.transform(test_pred[j])
        
        train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],train_pred_norm.shape[1])
        val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],val_pred_norm.shape[1])
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],test_pred_norm.shape[1])

##########################################################################################################################
        ncol = 6
        encoding_dim = 4
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        #encoder = Model(inputs = input_dim, outputs = encoded2)
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('MyModelCheckpoint/encoder_model.h5')
        train_pred_norm_auto=np.array(encoder.predict(train_pred_norm))
        val_pred_norm_auto=np.array(encoder.predict(val_pred_norm))
        test_pred_auto=np.array(encoder.predict(test_pred_norm))

        train_pred_norm_auto=pd.DataFrame(train_pred_norm_auto)
        val_pred_norm_auto=pd.DataFrame(val_pred_norm_auto)
        test_pred_auto=pd.DataFrame(test_pred_auto)                    
        
        

####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        tf.reset_default_graph()
        K.clear_session()
        
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        input_pred=Input(batch_shape=(None,4),name='input_pred')
        
        encoded12 = Dense(8, activation = 'relu')(input_pred)
        encoded22 = Dense(16, activation = 'relu')(encoded12)
        encoded32 = Dense(32, activation = 'relu')(encoded22)
        encoded32=Reshape((1,32))(encoded32)
        

        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_0=concatenate([branch_0,encoded32],axis=1)
        #branch_0=add([branch_0,encoded32])
        branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        #branch_1=add([branch_1,encoded32])
        branch_2 = Conv1D(128,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        #branch_2=add([branch_2,encoded32])
        
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.01))(branch_3)
        

        model=Model(inputs=[input_data,input_pred],outputs=net)
        if y==71:
            model.summary()
        callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF025.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.01))


        model.fit({'input_data':train_input,'input_pred':train_pred_norm_auto},y_train,validation_data=[[val_input,val_pred_norm_auto],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)

        model.load_weights('MyModelCheckpoint/CIF025.h5')
        
        pred=model.predict({'input_data':test_input, 'input_pred':test_pred_auto})
        pred=scaler3.inverse_transform(pred)#######################################################

        
        final_predictions[y,:num_test] = pred.reshape(num_test)
        final_test[y,:num_test]=test[-num_test:]

        AAA=smape(pred.reshape(num_test),current_test,num_test)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(num_test),current_test)
        Final_smape2[y]=AAA
        mse1+=mse(pred.reshape(num_test),current_test)
        pbar.update(1)

np.savetxt('MyModelCheckpoint/prediction025.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/72)
print("SMAPE2 (NZR approach)")
print(smape2/72)
print("MSE1:")
print(mse1/72)
print (sum(Final_smape2)/72)    
#print(smape1)
#print(smape2)

2019-08-31 22:49:24.335909


  0%|                                                                                           | 0/72 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  1%|█▏                                                                                 | 1/72 [00:02<03:07,  2.64s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  3%|██▎                                                                                | 2/72 [00:05<03:09,  2.71s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 32%|██████████████████████████▏                                                       | 23/72 [01:04<02:19,  2.85s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 33%|███████████████████████████▎                                                      | 24/72 [01:07<02:16,  2.85s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 35%|████████████████████████████▍                                                     | 25/72 [01:09<02:10,  2.78s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 64%|████████████████████████████████████████████████████▍                             | 46/72 [02:06<01:09,  2.66s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 65%|█████████████████████████████████████████████████████▌                            | 47/72 [02:09<01:07,  2.69s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 67%|██████████████████████████████████████████████████████▋                           | 48/72 [02:11<01:04,  2.67s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 96%|██████████████████████████████████████████████████████████████████████████████▌   | 69/72 [02:55<00:06,  2.10s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 97%|███████████████████████████████████████████████████████████████████████████████▋  | 70/72 [02:57<00:04,  2.04s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 99%|████████████████████████████████████████████████████████████████████████████████▊ | 71/72 [02:59<00:02,  2.05s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_pred (InputLayer)         (None, 4)            0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 8)            40          input_pred[0][0]                 
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 16)           144         dense_1[0][0]                    
__________________________________________________________________________________________________
input_data (InputLayer)         (None, 6, 1)         0                                            
__________________________________________________________________________________________________
dense_3 (D

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [03:01<00:00,  2.06s/it]


2019-08-31 22:52:25.680146
-----------------
SMAPE1: (DeepEX approach)
12.007143218087467
SMAPE2 (NZR approach)
12.007143218087467
MSE1:
5147501736041.623
12.007143218087467
